# **Mapping Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Load Data**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

prefixPath = '/content/drive/MyDrive/Colab Notebooks/BloodCell/';
path = prefixPath+'dataset2-master/images/';

batch_size = 16
height = 320
width = 240

train_datagen = ImageDataGenerator(rescale=1./255)
train_data = train_datagen.flow_from_directory(
        path + 'TRAIN',           
        target_size=(height, width),
        batch_size=batch_size,
        class_mode="sparse",
        shuffle=True)  

test_datagen = ImageDataGenerator(rescale=1./255)
validation_data = test_datagen.flow_from_directory(
        path+'TEST',
        target_size=(height, width),
        batch_size=batch_size,
        class_mode="sparse",
        shuffle=True)

evaluation_datagen = ImageDataGenerator(rescale=1./255)
evaluation_data = evaluation_datagen.flow_from_directory(
        path+'TEST_SIMPLE',
        target_size=(height, width),
        batch_size=batch_size,
        class_mode="sparse",
        shuffle=True)

Found 9957 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.
Found 71 images belonging to 4 classes.


In [ ]:
# len(train_data)
# print(len(validation_data[10][0]))
# print(validation_data[10][1][0])
# print(len(validation_data[0][0][0][0][0]))
# print(validation_data.next())

# **Create the model CNN**

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import BatchNormalization
from keras import backend as K
from keras import optimizers

K.set_image_data_format("channels_last") # Returns the default image dimension ordering convention
keras.backend.clear_session()

model = Sequential()

# Layer 1: Convolutional layer 1
model.add(Conv2D(128, (3, 3), padding="same",input_shape=(height, width,3),data_format="channels_last"))
model.add(BatchNormalization())
model.add(Activation("relu"))

#Layer 2: Pooling layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#Layer 3: Convolutional layer 2
model.add(Conv2D(128, (3, 3),padding="same"))
model.add(Activation("relu"))

#Layer 4: Pooling layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#Layer 5: Convolutional layer 3
model.add(Conv2D(192, (3, 3), padding="same"))
model.add(Activation("relu"))

#Layer 6: Pooling layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#Layer 7: Convolutional layer 4
model.add(Conv2D(192, (3, 3), padding="same"))
model.add(Activation("relu"))

#Layer 8: Pooling layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# #Layer 9: Convolutional layer 5
model.add(Conv2D(256, (3, 3),padding="same"))
model.add(Activation("relu"))

#Layer 10: Pooling layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#Layer 11: Convolutional layer 6
model.add(Conv2D(256, (3, 3),padding="same"))
model.add(Activation("relu"))

#Layer 12: Pooling layer
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

#Layer 13: Flatten layer
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512,activation = "relu"))
model.add(Dropout(0.2))

model.add(Dense(64,activation = "relu"))
model.add(Dense(4,activation = 'softmax')) # 4 outputs.

adam = tf.optimizers.Adam(lr=0.00001)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = adam, metrics=['accuracy'])

# CNN model brief veiw of each layer 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 320, 240, 128)     3584      
                                                                 
 batch_normalization (BatchN  (None, 320, 240, 128)    512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 320, 240, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 160, 120, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 160, 120, 128)     147584    
                                                                 
 activation_1 (Activation)   (None, 160, 120, 128)     0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(train_data, epochs=100, validation_data=validation_data)

Epoch 1/100
623/623 [==============================] - 2335s 4s/step - loss: 1.3575 - accuracy: 0.3171 - val_loss: 1.2433 - val_accuracy: 0.4230
Epoch 2/100
623/623 [==============================] - 55s 89ms/step - loss: 1.0971 - accuracy: 0.5066 - val_loss: 0.8897 - val_accuracy: 0.6132
Epoch 3/100
623/623 [==============================] - 55s 88ms/step - loss: 0.8275 - accuracy: 0.6426 - val_loss: 0.8121 - val_accuracy: 0.6068
Epoch 4/100
623/623 [==============================] - 55s 88ms/step - loss: 0.6469 - accuracy: 0.7195 - val_loss: 0.6371 - val_accuracy: 0.7346
Epoch 5/100
623/623 [==============================] - 55s 88ms/step - loss: 0.5002 - accuracy: 0.7899 - val_loss: 0.5878 - val_accuracy: 0.7423
Epoch 6/100
623/623 [==============================] - 55s 88ms/step - loss: 0.3997 - accuracy: 0.8364 - val_loss: 0.5417 - val_accuracy: 0.7941
Epoch 7/100
623/623 [==============================] - 55s 88ms/step - loss: 0.3276 - accuracy: 0.8677 - val_loss: 0.5260 - val_ac

In [ ]:
# Evaluate on Validation data
scores = model.evaluate(validation_data)
print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

156/156 [==============================] - 10s 64ms/step - loss: 1.1819 - accuracy: 0.8408
evaluate accuracy: 84.08%


# **Convert เพื่อไป Load ใช้งานบน Web Browser**

In [ ]:
!pip install tensorflowjs
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, prefixPath + "BloodCell_Model")